In [27]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd

class spotWrapper():
    def __init__(self):
        client_id = 'accd0aa479164ddcbf1cbf822512b80b'
        client_secret = '58bfc467435045e7b61c86fb03385729'
        client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
        self.spot = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
    def searchArtist(self, string):
        return self.spot.search(q=string, type="artist")
    def getAlbums(self, uri):
        return self.spot.artist_albums(uri, album_type="album")
    def getTrack(self, song, artist):
        return self.spot.search(q=str(song + ' ' + artist))
    def getAudioFeatures(self, song_id):
        return self.spot.audio_features(song_id)

In [28]:
spot = spotWrapper()
spot_artists = pd.read_excel("/Users/garcgabe/Desktop/SpotifyArtists.xlsx").drop("Unnamed: 0", axis=1)
artists_uri = spot_artists[['spotify_name','artist_uri']]


In [47]:
# title of each column
columns = ["album_uri", "album_name", "total_tracks", "release_date", 
           "artist_uris", "artist_names"]

# init empty lists to load then concat into DF
album_uri, album_name, total_tracks, release_date, artist_uris, artist_names  = ([] for i in range (6))
for i in range(0,len(artists_uri)-25):
    name_uri = artists_uri["artist_uri"][i]
    search_tree = spot.getAlbums(name_uri)['items']
    for j in range(0,len(search_tree)):
        search_tree_split = search_tree[j]
        all_artists, all_uris = ([] for x in range(2))
        number_of_artists = len(search_tree_split["artists"])
        for artist_num in range(0,number_of_artists):
            artist_name_in_list = search_tree_split["artists"][artist_num]["name"]
            try:
                artist_uri_in_list = spot_artists.loc[spot_artists["spotify_name"] == artist_name_in_list, "artist_uri"].iloc[0]
            except:
                artist_uri_in_list = 0
            all_uris.append(artist_uri_in_list)
            all_artists.append(artist_name_in_list)
        artist_names.append(all_artists)
        artist_uris.append(all_uris)
        album_uri.append(search_tree_split["uri"])
        album_name.append(search_tree_split["name"])
        total_tracks.append(search_tree_split["total_tracks"])
        release_date.append(search_tree_split["release_date"])
final_df = pd.concat([pd.Series(album_uri), pd.Series(album_name), pd.Series(total_tracks),
                                    pd.Series(release_date), pd.Series(artist_uris), pd.Series(artist_names)], 
                                    axis=1, keys=columns)
        
    
        



In [51]:
final_df[final_df.duplicated(["album_name", "total_tracks"], keep=False)].sort_values("release_date")


,album_uri,album_name,total_tracks,release_date,artist_uris,artist_names
299,spotify:album:1g3Ek21j6qDWt2CtravhrX,The Blueprint 3,15,2009-09-08,[spotify:artist:3nFkdlSjzX9mRTtwJOzDYB],[JAY-Z]
298,spotify:album:2CUT0104gySOIvqwtXeFsX,The Blueprint 3,15,2009-09-08,[spotify:artist:3nFkdlSjzX9mRTtwJOzDYB],[JAY-Z]
280,spotify:album:6klUp8sQyRXGuJhqZu4PG3,My Beautiful Dark Twisted Fantasy,13,2010-01-01,[spotify:artist:5K4W6rqBFWDnAN6FQUkS6x],[Kanye West]
278,spotify:album:20r762YmB5HeofjMCiPMLv,My Beautiful Dark Twisted Fantasy,13,2010-11-22,[spotify:artist:5K4W6rqBFWDnAN6FQUkS6x],[Kanye West]
271,spotify:album:2P2Xwvh2xWXIZ1OWY9S9o5,Watch The Throne (Deluxe),16,2011-08-08,"[spotify:artist:3nFkdlSjzX9mRTtwJOzDYB, spotif...","[JAY-Z, Kanye West]"
...,...,...,...,...,...,...
363,spotify:album:2vM2H6lyPfitmNZlc1Vrxv,Me vs. Myself (Deluxe),23,2022-12-12,[spotify:artist:31W5EY0aAly4Qieq6OFu6I],[A Boogie Wit da Hoodie]
344,spotify:album:4bJOIOKUAXFamjkGQwiVH6,Lil Durk Presents: Loyal Bros 2,23,2022-12-16,"[0, spotify:artist:3hcs9uc56yIGFCSy9leWe7]","[Only The Family, Lil Durk]"
343,spotify:album:4lXLULUDQvqV1yHtLW5fyL,Lil Durk Presents: Loyal Bros 2,23,2022-12-16,"[0, spotify:artist:3hcs9uc56yIGFCSy9leWe7]","[Only The Family, Lil Durk]"
99,spotify:album:43GclyRDmDm52SA342WoAm,I Am Music,18,2023-03-31,[spotify:artist:55Aa2cqylxrFIXC767Z865],[Lil Wayne]
